In [3]:
import json

with open("data/processed/chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

print(f"Loaded {len(chunks)} chunks")
print(chunks[0].keys())


Loaded 2684 chunks
dict_keys(['chunk_id', 'text', 'source', 'topic', 'page'])


## Embed the chunks

In [4]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")

embeddings = model.encode(
    [c["text"] for c in chunks],
    normalize_embeddings=True
)

embeddings = np.array(embeddings, dtype="float32")


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


## Indexing - Cosine similarity

In [5]:
import faiss

dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embeddings)

faiss.write_index(index, "index/faiss.index")